In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [8]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [31]:
list(model.children())

[Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1)),
 Linear(in_features=400, out_features=120, bias=True),
 Linear(in_features=120, out_features=84, bias=True),
 Linear(in_features=84, out_features=10, bias=True)]

In [11]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [12]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [16]:
outputs

[tensor([[[[1.0561, 0.6668, 0.6745,  ..., 0.5722, 0.0000, 0.0127],
           [0.8745, 0.0768, 0.7733,  ..., 0.5018, 0.5057, 0.1124],
           [0.3280, 1.4815, 0.9931,  ..., 0.4879, 0.8082, 0.0000],
           ...,
           [0.9269, 0.2777, 0.3518,  ..., 0.2698, 0.6597, 0.2854],
           [0.4831, 1.5054, 0.0951,  ..., 0.4610, 0.0821, 0.2882],
           [0.2531, 0.5212, 0.9145,  ..., 0.6537, 0.8418, 0.5353]],
 
          [[0.1583, 0.4015, 0.7518,  ..., 0.6117, 0.5626, 0.0000],
           [0.7851, 0.2447, 0.1577,  ..., 0.2595, 1.1069, 0.8404],
           [0.5936, 0.1765, 1.5786,  ..., 0.4047, 0.3134, 0.8127],
           ...,
           [0.4656, 0.8647, 0.5172,  ..., 0.6377, 0.4505, 0.0888],
           [0.8600, 2.2218, 1.1745,  ..., 1.2096, 0.3432, 0.5204],
           [1.0050, 0.2055, 0.7428,  ..., 0.6248, 0.8743, 0.2903]],
 
          [[0.4336, 0.3414, 1.2068,  ..., 0.4928, 0.3722, 0.7258],
           [0.5219, 1.1258, 0.4142,  ..., 1.0992, 0.0390, 1.1809],
           [1.0886, 0.08

In [17]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [25]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                a=0,
                                mode='fan_out',
                                nonlinearity='relu')
        if m.bias is not None:
            init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [26]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


